<a href="https://colab.research.google.com/github/soroushosanlou/DeepLearning/blob/main/Cats_vs_Dogs_tfdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"


68608000/68606236 [==============================] - 1s 0us/step


In [ ]:
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) 
    image = tf.image.random_crop(image, size=[150, 150, 3])
    image = tf.image.random_brightness(image, max_delta=0.5)

    return image

In [ ]:
train_dir

'/root/.keras/datasets/cats_and_dogs_filtered/train'

In [ ]:
!ls /root/.keras/datasets/cats_and_dogs_filtered/train

cats  dogs


In [ ]:
validation_dir

'/root/.keras/datasets/cats_and_dogs_filtered/validation'

In [ ]:
! ls /root/.keras/datasets/cats_and_dogs_filtered/validation

cats  dogs


In [ ]:
train_dir = pathlib.Path(train_dir)          #ساخت پوشه آموزش با کتابخانه پت لیب

In [ ]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')]) 

In [ ]:
CLASS_NAMES

array(['dogs', 'cats'], dtype='<U4')

In [ ]:
full_dataset = tf.data.Dataset.list_files(str(train_dir/'*/*'))  


In [ ]:
for i in full_dataset.take(10):
  print(i)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.977.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.295.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.344.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.595.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.629.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.482.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.770.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.38.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.372.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs

In [ ]:
validation_split = 0.2            

In [ ]:
DATASET_SIZE = len(list(full_dataset))        
print("Dataset size: ", DATASET_SIZE)
train_size = int((1-validation_split) * DATASET_SIZE)     
print("train size: ", train_size)
train_dataset = full_dataset.take(train_size)   
validation_dataset = full_dataset.skip(train_size) 
print("Validation size: ", len(validation_dataset))

Dataset size:  2000
train size:  1600
Validation size:  400


In [ ]:
def get_label(file_path):           
  parts = tf.strings.split(file_path, os.path.sep)      
  return parts[-2] == CLASS_NAMES

In [ ]:
get_label("/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.949.jpg").numpy()

array([ True, False])

In [ ]:
def load_img(image_path):
    img = tf.io.read_file(image_path)
    
    img = tf.image.decode_image(img, 3, expand_animations=False)   
                                                                                       
    img = tf.cast(img, tf.float32)          
    return img

In [ ]:
def normalize(image):                  
    image = (image / 127.5) - 1    
    return image

In [ ]:
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

In [ ]:
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [ ]:
def load_image_train(image_file):                 
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image = normalize(image)
    
    return image, label

In [ ]:
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)

    return image, label

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE


In [ ]:
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
train_dataset = train_dataset.map(load_image_train)        
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)     
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False        

In [ ]:
n_class =len(CLASS_NAMES)
n_class

2

In [ ]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(100, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(n_class, activation='sigmoid')

In [ ]:
model = tf.keras.Sequential([
  base_model,
  flatten_layer,
  dense_layer,
  dropout_layer,
  prediction_layer
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=validation_dataset)

Epoch 1/100
50/50 [==============================] - 45s 167ms/step - loss: 0.7597 - accuracy: 0.7219 - val_loss: 0.4048 - val_accuracy: 0.8325
Epoch 2/100
50/50 [==============================] - 10s 134ms/step - loss: 0.3742 - accuracy: 0.8238 - val_loss: 0.3399 - val_accuracy: 0.8625
Epoch 3/100
50/50 [==============================] - 10s 137ms/step - loss: 0.3534 - accuracy: 0.8356 - val_loss: 0.2977 - val_accuracy: 0.8825
Epoch 4/100
50/50 [==============================] - 10s 135ms/step - loss: 0.3057 - accuracy: 0.8587 - val_loss: 0.4550 - val_accuracy: 0.8375
Epoch 5/100
50/50 [==============================] - 10s 137ms/step - loss: 0.3140 - accuracy: 0.8500 - val_loss: 0.6954 - val_accuracy: 0.7400
Epoch 6/100
50/50 [==============================] - 10s 133ms/step - loss: 0.3015 - accuracy: 0.8669 - val_loss: 0.3454 - val_accuracy: 0.8775
Epoch 7/100
50/50 [==============================] - 10s 134ms/step - loss: 0.2829 - accuracy: 0.8581 - val_loss: 0.4985 - val_accuracy: